In [1]:
import os
import gc
import dask
import dask.array as da
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster, progress
gc.collect()

os.chdir(
    "/mnt/cloud/wwu1/ec_bronze/_nogroup/ae78a1ca-a0e8-4e4e-8992-69c34947db65/UseCase_AIRCON"
)


# Define a custom function to calculate wind speed
def wind_speed(u, v):
    return da.sqrt(u**2 + v**2)


def wind_dir(u, v):
    return (270 - da.arctan2(u, v) * 180 / np.pi) % 360


scale_factor_wd = 360 / (2**16 - 1)  # applies to all datasets

# Define your open_mfdataset parameters
chunks = {"longitude": "auto", "latitude": "auto", "time": "auto"}


In [3]:
client = Client()
print(client)

<Client: 'tcp://127.0.0.1:45485' processes=4 threads=8, memory=32.00 GiB>


In [52]:
y = 2015
# Open the datasets using xarray.open_mfdataset() and Dask
d = xr.open_mfdataset(
    [
        f"supplementary/era5/ERA5_10_metre_U_wind_component_{y}.nc",
        f"supplementary/era5/ERA5_10_metre_V_wind_component_{y}.nc",
    ],
    chunks=chunks,
    combine="by_coords",
    parallel=True,
)  # .isel(time=range(0,64))

d

<xarray.Dataset>
Dimensions:    (longitude: 663, latitude: 460, time: 8760)
Coordinates:
  * longitude  (longitude) float32 -30.63 -30.53 -30.43 ... 35.37 35.47 35.57
  * latitude   (latitude) float32 71.66 71.56 71.46 71.36 ... 25.96 25.86 25.76
  * time       (time) datetime64[ns] 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(2053, 106, 154), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(2053, 106, 154), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-10-11 21:09:15 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [54]:
%%time

d["ws"] = wind_speed(d.u10, d.v10)
mini = d.ws.min().compute().values.item()
maxi = d.ws.max().compute().values.item()
scale_factor_ws = (maxi - (-mini)) / (2**16 - 1)
print(scale_factor_ws)

0.0004245024505094613
CPU times: user 45.4 s, sys: 13.6 s, total: 58.9 s
Wall time: 6min 57s


In [55]:
out_ws = f"supplementary/wind_test/ERA5_10_metre_Wind_Speed_{y}.nc"
ws = (
    d["ws"]
    .to_dataset()
    .to_netcdf(
        out_ws,
        compute=False,
        encoding={
            "ws": {
                "dtype": "int16",
                "scale_factor": scale_factor_ws,
                "_FillValue": -9999,
            }
        },
    )
)

In [56]:
# ws_res = client.gather(client.persist(ws))
ws_res = ws.persist()
progress(ws_res)

VBox()

In [57]:
out_wd = f"supplementary/wind_test/ERA5_10_metre_Wind_Direction_{y}.nc"
d["wd"] = wind_dir(d.u10, d.v10)
wd = (
    d["wd"]
    .to_dataset()
    .to_netcdf(
        out_wd,
        compute=False,
        encoding={
            "wd": {
                "dtype": "int16",
                "scale_factor": scale_factor_wd,
                "_FillValue": -9999,
            }
        },
    )
)

In [58]:
wd_res = wd.persist()
progress(wd_res)

VBox()

In [59]:
d.close()

In [43]:
r = xr.open_dataset(out_ws)
r

<xarray.Dataset>
Dimensions:    (longitude: 663, latitude: 460, time: 8760)
Coordinates:
  * longitude  (longitude) float32 -30.63 -30.53 -30.43 ... 35.37 35.47 35.57
  * latitude   (latitude) float32 71.66 71.56 71.46 71.36 ... 25.96 25.86 25.76
  * time       (time) datetime64[ns] 2017-01-01 ... 2017-12-31T23:00:00
Data variables:
    ws         (time, latitude, longitude) float32 ...

In [ ]:
# client.close()